In [1]:
from __future__ import unicode_literals
from matplotlib import pyplot as plt
import pickle
import pandas as pd
import numpy as np
import nltk as nk
import untangle
import seaborn as sns
import operator
import string 
nk.download('punkt')
nk.download('stopwords')
nk.download('wordnet')
nk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/andres/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/andres/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/andres/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/andres/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Asociación de palabras

#### 1. Levantar el corpus AP, separando cada noticia como un elemento distinto en un diccionario (DOCNO :TEXT).


In [2]:
corpus_ap = {}
obj = untangle.parse('ap/ap.txt')
for i in range(len(obj.root.DOC)):
    documento = obj.root.DOC[i]
    corpus_ap[documento.DOCNO.cdata.strip()] = documento.TEXT.cdata.strip()

### 2. Calcular el tamaño del vocabulario

#### Pasos de filtrado

1. Tokenizamos oraciones
2. Para cada oración realizamos una tokenización de palabras básica (separamos por espacios)
3. Computamos el POS tag de cada palabra
4. Usando el POS tag lematizamos cada palabra
5. Para cada palabra lematizada aumentamos en 1 la cantidad de veces que aparece

In [17]:
sentences = []

for (docno, text) in corpus_ap.items():
    tokens = nk.sent_tokenize(text.lower())
    for sent in tokens:
        sentences.append(sent)
len(sentences)

def clean_chars(word):
    replaceable = [",", ".", "-", "_", "`", ":", ";", "!", "?", "`", "'"]
    for token in replaceable:
        word = word.replace(token, " ")
    return word.strip()
    
words_by_sent = [[clean_chars(w) for w in s.split() if len(clean_chars(w)) > 1 or clean_chars(w).isalpha()] 
                 for s in sentences]

In [18]:
word_semantic = nk.tag.pos_tag_sents(words_by_sent)

In [19]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return nk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nk.corpus.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nk.corpus.wordnet.ADV
    else:
        return nk.corpus.wordnet.NOUN



lemmatizer = nk.WordNetLemmatizer()

word_count = {}
for sentence in word_semantic:    
    for (word, tag) in sentence:
        lemmatized = lemmatizer.lemmatize(word, get_wordnet_pos(tag))
        word = lemmatized.split()[0]
        if not word[0].isalpha():
            continue
        elif word not in word_count:
            word_count[word] = 1
        else:
            word_count[word] += 1
            
with open('word_count.pickle', 'wb') as handle:
    pickle.dump(word_count, handle, protocol=pickle.HIGHEST_PROTOCOL)

{u'fawn': 1,
 u'circuitry': 3,
 u'illsley': 1,
 u'darryls': 1,
 u'francesco': 3,
 u'woody': 8,
 u'regularize': 1,
 u'chatter': 1,
 u'seveny': 1,
 u'scold': 2,
 u'refunding': 2,
 u'sevene': 1,
 u'hermann': 7,
 u'vazgen': 1,
 u'cullom': 1,
 u'stipulate': 7,
 u'pigment': 1,
 u'appropriation': 31,
 u'heublein': 1,
 u'broward': 4,
 u'bringing': 3,
 u'wooded': 4,
 u'grueling': 4,
 u'broiler': 3,
 u'wooden': 16,
 u'wednesday': 699,
 u'woods': 1,
 u'steadfastness': 1,
 u'cichan': 1,
 u'hanging': 4,
 u'kopetski': 1,
 u'feasibility': 1,
 u'rudman': 12,
 u'gorman': 2,
 u'sustaining': 1,
 u'skadden': 1,
 u'inanimate': 1,
 u'centimeter': 2,
 u'cooking': 7,
 u'numeral': 1,
 u'succumb': 4,
 u'presentence': 1,
 u'crouch': 6,
 u'alpena': 1,
 u'ching': 3,
 u'china': 153,
 u'affiliated': 6,
 u'chino': 1,
 u'chinn': 19,
 u'chink': 1,
 u'doldrums': 4,
 u'reggae': 3,
 u'ansgar': 1,
 u'dekoven': 1,
 u'paulucci': 1,
 u'controversy': 31,
 u'nepalganj': 1,
 u'spotty': 3,
 u'climber': 3,
 u'appropriately': 3,
 u

In [29]:
stopw = set(nk.corpus.stopwords.words("english"))

with open('word_count.pickle', 'rb') as handle:
    word_count = pickle.load(handle) 

# Aca limpiamos las stopwords
word_count = [(x, y) for (x, y) in sorted(word_count.items(), key=operator.itemgetter(1), reverse=True)
                                                 if x not in stopw and len(x) > 1]

In [34]:
print "Tamaño del vocabulario: ", len(word_count), " palabras."

Tamaño del vocabulario:  27291  palabras.


#### 3. Para las 500 palabras con más apariciones, calcular el par más asociado según la medida presentada.

In [32]:
most_used = word_count[:500]

Para el cálculo utilizaremos una ventana de 5 palabras como indica el paper.

In [ ]:
max_assoc = []

for i in range(len(obj.root.DOC)):
    documento = obj.root.DOC[i]
    for word in most_used:
        asociaciones = {}
        

## Información Léxica[2]
Bajar de Project Gutenberg el libro de Darwin ON THE ORIGIN OF SPECIES.

#### 1. Procesar el texto, tokenizando eliminando signos de puntuación.

#### 2. Siguiendo el artículo de la sección, calcular la autocorrelación para estimar la distribución de la palabra a lo largo del texto.

#### 3. Armar una función que reciba una lista de tokens, una lista de palabras y un tamaño de ventana y devuelva una lista de probabilidades de encontrar la palabra en cada ventana para cada palabra pasada por parámetro.

#### 4. Calcular la entropía de la distribución de palabras seleccionadas para distintos tamaños de ventana

#### 5. Generar una versión randomizada del texto, y medir la entropía de las palabras randomizadas.

#### 6. Distinguir las palabras del texto en art´ıculos, sustantivos y adjetivos usando un POS-tagger. Verificar si las medidas separan a estos grupos de palabras.

## Word embeddings, distancia semántica y WordNet

#### 1. Utilizando el test WordSim3531, comparar el rendimiento entre LSA[3] y Word2Vec2

#### 2. Comparar los distintos word embeddings con las medidas definidas en WordNet.